# Get same date sentinel-1 and sentinel-2 images

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import geemap
import ee
import rasterio
import contextlib
from p_tqdm import p_map

In [2]:
ee.Authenticate()
ee.Initialize(project="ee-sarice")

# Import datasets

In [3]:
filterSizeMeters = 60 # speckle filter size, in meter
scale = 30 # export image resolution, in meter

In [4]:
def filterSpeckles(img):
    # Function provided by Abhishek Kumar
    vv = img.select("VV")
    # spf: speckle filtered
    vv_filtered = vv.focal_median(filterSizeMeters, 'circle','meters').rename("VV_spf")
    return img.addBands(vv_filtered)

In [5]:
def download_s1s2_sameday_images(lake_id, # export 60 meter resolution
                           ):
    # hydrolakes
    hydrolakes = ee.FeatureCollection("projects/ee-lakeice/assets/HydroLAKES_polys_v10")        
    aoi = hydrolakes.filterMetadata('Hylak_id', 'equals', lake_id)
    roi = aoi.geometry()
    
    # Load SENTINEL-1
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
            .filterBounds(roi)
    # speckle filter
    # add image date to the property
    s1_with_dates = s1.map(lambda image: filterSpeckles(image).set('date', 
                                                   ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss')))
    
    # SENTINEL-2
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(roi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',15))

    s2_with_dates = s2.map(lambda image: image.set('date',
                                                    ee.Date(image.get('system:time_start')).format('YYYY-MM-dd HH:mm:ss')))
    
    # get dates
    s1_dates = s1_with_dates.aggregate_array("date").getInfo()
    s2_dates = s2_with_dates.aggregate_array("date").getInfo()
    s1_dates = pd.to_datetime(s1_dates)
    s2_dates = pd.to_datetime(s2_dates)
    
    # Find sentinel-2 image that was acquired < 12 hour to the sentinel-1 image
    def lookfor_sen2(sen1_date, 
                     sen2_dates = s2_dates, # list of dates of avaialble sentinel-2 images
                     delta_hr = 12):
        # find all times that has less than 12 hour interval to the sentinel-2 image
        intervals = np.abs(sen2_dates - sen1_date)
        indexes = intervals < pd.Timedelta(f"{delta_hr}H")

        # get the dates
        match_sen2_date = sen2_dates[indexes]

        if len(match_sen2_date) == 0:
            return None
        else:
            return match_sen2_date
        
    # create a match table between sentinel-1 dates and sentinel-2 dates
    match_table = pd.DataFrame(s1_dates, columns = ["s1_dates"])
    match_table["s2_dates"] = match_table["s1_dates"].apply(lookfor_sen2)
    match_table = match_table.dropna().reset_index(drop = True).explode("s2_dates")

    # Find Sentinel-1
    s1_date_string = ee.List(list(match_table.s1_dates.apply(lambda date: date.strftime('%Y-%m-%d %X')).unique()))
    s2_date_string = ee.List(list(match_table.s2_dates.apply(lambda date: date.strftime('%Y-%m-%d %X')).unique()))
    
    # create image collections of the sentinel-1, sentinel-2 (SCL)
    matched_s1_img_col = s1_with_dates.filter(ee.Filter.inList('date',s1_date_string)).select(["VV_spf", "VH", "angle"])
    matched_s2_img_col = s2_with_dates.filter(ee.Filter.inList('date',s2_date_string)).select(["B4", "B3", "B2", "SCL"])
      
    # directory to save the s1 and s2 images
    out_dir_s1 = f"/mnt/Data_2tb/sentinel1_ice/sen1/{lake_id}"
    out_dir_s2 = f"/mnt/Data_2tb/sentinel1_ice/sen2/{lake_id}"
    
    try:
        os.mkdir(out_dir_s1)
        os.mkdir(out_dir_s2)
    except:
        pass
    
    print(f"Downloading Sentinel-2 RGBS Image for {lake_id}")
    geemap.download_ee_image_collection(matched_s2_img_col, 
                                    out_dir = out_dir_s2,
                                    region = roi,
                                    dtype = "float32",
                                    num_threads = 2,
                                    scale = scale,
                                    )
    
    # Download Sentinel-1 imagery
    #print(f"Downloading Sentinel-1 Image for {lake_id}")
    #geemap.download_ee_image_collection(matched_s1_img_col, 
    #                                out_dir = out_dir_s1,
    #                                region = roi,
    #                                dtype = "float32",
    #                                num_threads = 2,
    #                                scale = scale,
    #                                )
    
    return match_table

# Retrieve images for all training lakes

In [6]:
lake_ids = pd.read_csv("data/training_lakes.csv").Hylak_id.to_numpy().tolist()

In [7]:
len(lake_ids)

1360

In [ ]:
for lake_id in lake_ids:
    download_s1s2_sameday_images(lake_id)
    break

Total number of images: 193



20190117T070221_20190117T070222_T40TEM.tif: |                       | 0.00/110M (raw) [  0.0%] in 00:00 (eta: …